In [3]:
import requests
import pandas as pd
import time

api_key = 'bf097708532e1fdb7a8509ff205604d3'
# CHECAR A VERSÂO DO API, nesse caso, estava na versão 3, indicada ao fim do url
base_url = 'https://api.themoviedb.org/3/'

#  Obter a lista de Gêneros 
# Usado para não cadastrar os gêneros de maneira manual e correr o risco de deixar de incluir algum.
endpoint_genres = f'{base_url}genre/movie/list'
params_genres = {'api_key': api_key, 'language': 'pt-BR'}
response_genres = requests.get(endpoint_genres, params=params_genres)
genres_data = response_genres.json()

genre_map = {genre['id']: genre['name'] for genre in genres_data.get('genres', [])}
print(f"Mapeamento de Gêneros carregado: {genre_map}")

#  Obter Filmes Populares (ou outro filtro pré-existente no site)
endpoint_popular_movies = f'{base_url}movie/popular' # Ou 'movie/top_rated', 'movie/now_playing'
params_popular = {
    'api_key': api_key,
    'language': 'pt-BR',
    'page': 1
}

all_movies_data = []
num_pages_to_fetch = 50 #Aqui fica o limite de páginas procuradas - limite da página é 40 solicitações a cada segundo! Fetch testado até 50 funcional.
# Se algum erro ocorrer em futuras inspeções, aumentar o tempo de Sleep entre os processos.
# Programa rodado dia 19/06/25 - 20 filmes por página na configuração atual
print(f"\nIniciando a busca por {num_pages_to_fetch} páginas de filmes populares...")

for page_num in range(1, num_pages_to_fetch + 1):
    params_popular['page'] = page_num
    response_popular = requests.get(endpoint_popular_movies, params=params_popular)
    popular_movies_data = response_popular.json()

    if 'results' in popular_movies_data:
        print(f"Processando filmes da página {page_num}...")
        for movie_basic in popular_movies_data['results']:
            movie_id = movie_basic.get('id')
            if not movie_id:
                continue

            # Buscar Detalhes Completos do Filme
            endpoint_movie_details = f'{base_url}movie/{movie_id}'
            params_details = {'api_key': api_key, 'language': 'pt-BR'}
            response_details = requests.get(endpoint_movie_details, params=params_details)
            movie_full_details = response_details.json()

            # Extrair nomes dos gêneros
            genres = [g['name'] for g in movie_full_details.get('genres', [])]
            genres_str = ', '.join(genres) if genres else None

            # Buscar Créditos do Filme para primeiro diretor e primeiro roteirista 
            endpoint_credits = f'{base_url}movie/{movie_id}/credits'
            params_credits = {'api_key': api_key}
            response_credits = requests.get(endpoint_credits, params=params_credits)
            credits_data = response_credits.json()

            director = None
            screenwriter = None # Ou 'Writer' para roteirista

            for crew_member in credits_data.get('crew', []):
                if crew_member.get('job') == 'Director':
                    director = crew_member.get('name')
                if crew_member.get('job') == 'Screenplay' or crew_member.get('job') == 'Writer':
                    if screenwriter is None:
                        screenwriter = crew_member.get('name')
                if director and screenwriter:
                    break
            
        
            all_movies_data.append({
                'id_filme': movie_basic.get('id'),
                'titulo': movie_basic.get('title'),
                'data_lancamento': movie_basic.get('release_date'),
                'media_avaliacoes': movie_basic.get('vote_average'),
                'numero_avaliacoes': movie_basic.get('vote_count'),
                'popularidade': movie_basic.get('popularity'),
                'sinopse': movie_basic.get('overview'),
                'idioma_original': movie_basic.get('original_language'),
                'generos': genres_str,
                'diretor': director,
                'roteirista': screenwriter
            })
            
            time.sleep(0.05)
    else:
        print(f"Erro ao buscar dados na página {page_num}: {popular_movies_data.get('status_message', 'Erro desconhecido')}")
        break
    
    time.sleep(0.5) # Atraso maior entre as páginas


df_movies_detailed = pd.DataFrame(all_movies_data)


output_filename = 'tmdb_movies_detailed.csv'
df_movies_detailed.to_csv(output_filename, index=False, encoding='utf-8')
# Comédia - Com#dia - Ação - A&%o
print(f"\nDados detalhados de {len(df_movies_detailed)} filmes salvos em '{output_filename}'")

Mapeamento de Gêneros carregado: {28: 'Ação', 12: 'Aventura', 16: 'Animação', 35: 'Comédia', 80: 'Crime', 99: 'Documentário', 18: 'Drama', 10751: 'Família', 14: 'Fantasia', 36: 'História', 27: 'Terror', 10402: 'Música', 9648: 'Mistério', 10749: 'Romance', 878: 'Ficção científica', 10770: 'Cinema TV', 53: 'Thriller', 10752: 'Guerra', 37: 'Faroeste'}

Iniciando a busca por 50 páginas de filmes populares...
Processando filmes da página 1...
Processando filmes da página 2...
Processando filmes da página 3...
Processando filmes da página 4...
Processando filmes da página 5...
Processando filmes da página 6...
Processando filmes da página 7...
Processando filmes da página 8...
Processando filmes da página 9...
Processando filmes da página 10...
Processando filmes da página 11...
Processando filmes da página 12...
Processando filmes da página 13...
Processando filmes da página 14...
Processando filmes da página 15...
Processando filmes da página 16...
Processando filmes da página 17...
Proces